In [75]:
"""Constructed a NN model that performs binary weather classification to determine if it is going to
rain or not. The test accuracy of the model is 100 percent accuracy
(without use of a validation set). This model does not require the use of regularization/dropout.
"""
import numpy as np
import pandas as pd
import math

import os
import random

import torch
from torch import nn
import torch.optim as optim

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


# Random seed for training/testing.
RAND_SEED = 42
random.seed(RAND_SEED)
np.random.seed(RAND_SEED)
torch.random.manual_seed(RAND_SEED)


def get_cleaned_dataset(path_to_dataset):
    df = pd.read_csv(path_to_dataset)

    # Remove biased based features.
    df = df.drop(
        columns=[
            "Date",
            "Location",
            "RainTomorrow",
            "Evaporation",
            "Sunshine",
            "WindGustDir",
            "WindDir9am",
            "WindDir3pm",
        ],
    )

    # Convert RainToday from string feature to interger feature with 1
    # representing it will rain and 0 representing that it will not rain.
    df["RainToday"] = df["RainToday"].replace(["No", "Yes"], [0, 1])

    # Remove any record where a feature is na. There is enough data where
    # we shouldn't bother replacing with the mean of the column.
    df = df.dropna()

    return df


def get_scaled_train_test_datasets(df):
    # Split dataset into train/test.
    # ...RainToday is our target vector.
    y = df[["RainToday"]]
    X = df.drop(columns=["RainToday"])

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=RAND_SEED, shuffle=True
    )

    # Scale input data to to make finding the gradient significantly faster.
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    
    # Convert data to the form that pytorch requires (float32 tensors).
    X_train = torch.from_numpy(X_train.astype(np.float32))
    X_test = torch.from_numpy(X_test.astype(np.float32))
    y_train = torch.from_numpy(y_train.to_numpy().astype(np.float32))
    y_test = torch.from_numpy(y_test.to_numpy().astype(np.float32))

    y_train = y_train.view(y_train.shape[0], 1)
    y_test = y_test.view(y_test.shape[0], 1)

    return X_train, X_test, y_train, y_test


class Network(nn.Module):
    """NN for detecting if it is going to rain or not.
    This network is strictly used for binary classification.
    Effectively what we are doing is Logistic Regression with
    no regularization. This NN has 1 input layer, 2 hidden, and one output.
    """

    def __init__(self, input_size):
        super(Network, self).__init__()

        self.hidden = nn.Linear(input_size, input_size)
        self.hidden2 = nn.Linear(input_size, input_size)
        self.output = nn.Linear(input_size, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.hidden(x)  
        x = self.relu(x)

        x = self.hidden2(x)
        x = self.relu(x)

        x = self.output(x)
        x = self.sigmoid(x)
        return x


def main():
    # Set training device.
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Get dataset.
    weather_dataset_url = '/content/rain.csv'
    X_train, X_test, y_train, y_test = get_scaled_train_test_datasets(
        get_cleaned_dataset(path_to_dataset=weather_dataset_url)
    )

    # Decide input and output size for NN.
    # ...input size is the number of features (Columns in X_train/X_test).
    # ... output size is 1 because we are doing logistic regression.
    input_size = X_train.shape[1]

    # Set hyper params for NN.
    learning_rate = 0.01
    number_of_epochs = 190

    # Create network.
    model = Network(input_size=input_size).to(device)

    # Set loss function.
    loss_function = nn.BCELoss()

    # Set optimization function for finding gradient.
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    # Train network.
    for epoch in range(number_of_epochs):
        # Forward pass and loss calculation.
        y_predicted = model(X_train)
        loss = loss_function(y_predicted, y_train)

        # Backwards propagation pass.
        loss.backward()

        # Update weights
        optimizer.step()

        # Zero-out/empty gradients before next iteration.
        optimizer.zero_grad()

        # Display the loss of the function every 10 steps.
        if epoch + 1 % 10 == 0:
            print(f"epoch: {epoch + 1}, loss = {loss.item():.4f}")

    # Test model and get accuracy.
    with torch.no_grad():
        # Get accuracy of model.
        # ... To determine if something is class 1, we will simply use 0.5
        # as the cutoff point for our Logistic Regression.
        y_predicted = model(X_test)
        y_predicted_classes = y_predicted.round()
        accuracy = y_predicted_classes.eq(y_test).sum() / float(y_test.shape[0])
        print(f"Model accuracy: {accuracy:.4f}")
    
    # Save the model now that testing is complete.
    torch.save(model, f=os.path.join(os.getcwd(), 'model.pt'))


if __name__ == "__main__":
    main()


Model accuracy: 1.0000
